In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold

import tensorflow as tf
cpu_count()

4

In [2]:
!pip freeze > requirements.txt
!cat requirements.txt

import sys
print(f"Python version: {sys.version}")

absl-py==1.4.0
accelerate==0.12.0
access==1.1.8
affine==2.4.0
aiobotocore==2.4.2
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1663850771047/work
aiohttp-cors==0.7.0
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
albumentations==1.3.0
alembic==1.9.4
altair==4.2.2
annoy==1.17.1
ansiwrap==0.8.4
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1666191106763/work/dist
apache-beam==2.44.0
aplus==0.11.0
appdirs==1.4.4
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1649500320262/work
arrow==1.2.3
arviz==0.12.1
astroid==2.14.2
astropy==4.3.1
astunparse==1.6.3
async-timeout @ file:///home/conda/feedstock_root/build_artifacts/async-timeout_1640026696943/work
asynctest==0.13.0
atpublic==2.3
attrs @ file:///home/conda/feedstock_root/build_ar

In [3]:
train_df = pd.read_csv('/kaggle/input/australian-fingerspelling/train.csv')
train_df

,path,file_id,sequence_id,participant_id,phrase
0,landmarks/101.parquet,101,10101,1,abcdefghijklmnopqrstuvwxyz
1,landmarks/101.parquet,101,10104,1,antecedent
2,landmarks/101.parquet,101,10105,1,antifascist
3,landmarks/101.parquet,101,10106,1,cooperate
4,landmarks/101.parquet,101,10107,1,describe
...,...,...,...,...,...
3928,landmarks/103.parquet,103,31492,4,nunawading
3929,landmarks/103.parquet,103,31493,4,somalia
3930,landmarks/103.parquet,103,31494,4,tokyo
3931,landmarks/103.parquet,103,31495,4,yarra


In [4]:
temp = pd.read_parquet('/kaggle/input/australian-fingerspelling/landmarks/101.parquet')
temp

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
10101,0,0.386425,0.386046,0.386145,0.383024,0.385855,0.385589,0.384911,0.364171,0.384473,...,-0.009946,-0.010004,-0.007199,-0.009539,-0.006974,-0.005153,-0.009149,-0.009066,-0.005984,-0.003608
10101,1,0.387526,0.386954,0.387151,0.383922,0.386737,0.386461,0.385760,0.365443,0.385282,...,-0.016328,-0.018092,-0.011789,-0.016041,-0.014370,-0.012743,-0.015030,-0.016318,-0.013779,-0.011740
10101,2,0.388560,0.387876,0.388139,0.384882,0.387642,0.387376,0.386746,0.366467,0.386280,...,-0.023372,-0.021855,-0.017763,-0.023876,-0.019103,-0.014390,-0.022050,-0.023017,-0.017505,-0.013240
10101,3,0.389577,0.388640,0.388989,0.385550,0.388371,0.388074,0.387399,0.366969,0.386924,...,-0.020876,-0.019681,-0.016725,-0.021370,-0.016427,-0.012090,-0.020847,-0.021081,-0.015429,-0.011245
10101,4,0.389694,0.388630,0.389059,0.385712,0.388375,0.388131,0.387623,0.367540,0.387222,...,-0.019346,-0.018175,-0.015666,-0.020022,-0.015669,-0.011632,-0.019722,-0.019658,-0.014135,-0.009925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11496,62,0.428776,0.428433,0.428936,0.426428,0.428420,0.428636,0.429445,0.409080,0.429872,...,-0.027019,-0.027143,-0.019228,-0.027176,-0.024285,-0.020927,-0.024493,-0.028176,-0.024733,-0.021397
11496,63,0.427878,0.427578,0.428041,0.425548,0.427577,0.427788,0.428584,0.408051,0.429021,...,-0.026828,-0.026971,-0.019031,-0.026978,-0.024108,-0.020774,-0.024329,-0.027973,-0.024508,-0.021169
11496,64,0.426954,0.426578,0.427079,0.424538,0.426577,0.426793,0.427616,0.406903,0.428053,...,-0.025417,-0.025310,-0.017889,-0.025822,-0.023131,-0.019803,-0.022876,-0.026677,-0.023391,-0.020054


In [5]:
temp.loc[temp.index==10101]

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
10101,0,0.386425,0.386046,0.386145,0.383024,0.385855,0.385589,0.384911,0.364171,0.384473,...,-0.009946,-0.010004,-0.007199,-0.009539,-0.006974,-0.005153,-0.009149,-0.009066,-0.005984,-0.003608
10101,1,0.387526,0.386954,0.387151,0.383922,0.386737,0.386461,0.385760,0.365443,0.385282,...,-0.016328,-0.018092,-0.011789,-0.016041,-0.014370,-0.012743,-0.015030,-0.016318,-0.013779,-0.011740
10101,2,0.388560,0.387876,0.388139,0.384882,0.387642,0.387376,0.386746,0.366467,0.386280,...,-0.023372,-0.021855,-0.017763,-0.023876,-0.019103,-0.014390,-0.022050,-0.023017,-0.017505,-0.013240
10101,3,0.389577,0.388640,0.388989,0.385550,0.388371,0.388074,0.387399,0.366969,0.386924,...,-0.020876,-0.019681,-0.016725,-0.021370,-0.016427,-0.012090,-0.020847,-0.021081,-0.015429,-0.011245
10101,4,0.389694,0.388630,0.389059,0.385712,0.388375,0.388131,0.387623,0.367540,0.387222,...,-0.019346,-0.018175,-0.015666,-0.020022,-0.015669,-0.011632,-0.019722,-0.019658,-0.014135,-0.009925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10101,135,0.383998,0.382508,0.383329,0.379550,0.382106,0.381840,0.381365,0.362493,0.380763,...,-0.010713,-0.012136,-0.008560,-0.011668,-0.013466,-0.014682,-0.011466,-0.013561,-0.014022,-0.014291
10101,136,0.383834,0.382402,0.383192,0.379462,0.382007,0.381748,0.381272,0.362402,0.380671,...,-0.010206,-0.011594,-0.008517,-0.011090,-0.012584,-0.013662,-0.011421,-0.012777,-0.012758,-0.012745
10101,137,0.383521,0.382033,0.382846,0.379019,0.381616,0.381327,0.380776,0.361986,0.380128,...,-0.010549,-0.012362,-0.008295,-0.011491,-0.013075,-0.014227,-0.011448,-0.013732,-0.014215,-0.014545


In [6]:
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [7]:
with open('/kaggle/input/australian-fingerspelling/character_to_prediction_index.json') as json_file:
    LABEL_DICT = json.load(json_file)

In [9]:
selected_columns = temp.drop(columns=['frame']).columns
selected_columns

def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=selected_columns)

def encode_strings(string_list, mapping=LABEL_DICT):
    return [[mapping[char] for char in string] for string in string_list]

In [10]:
def encode_row(row):
    pq_path = f'/kaggle/input/australian-fingerspelling/{row.path}'
    coordinates = load_relevant_data_subset(pq_path)
    
    if coordinates.empty:
        print(f"No coordinates loaded for sequence_id {row.sequence_id} in {pq_path}.")
    
    coordinates = coordinates.loc[coordinates.index.astype(str)==str(row.sequence_id)].values.astype('float32')
    
    if coordinates.size == 0:
        print(f"No matching coordinates found for sequence_id {row.sequence_id} in {pq_path}.")
        
    coordinates_encoded = coordinates.tobytes()
    participant_id = int(row.participant_id)
    sequence_id = int(row.sequence_id)
    phrase = row.phrase.encode('utf-8')
    phrase_encoded = encode_strings([row.phrase])[0]
    
    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
                'file_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
                'phrase': tf.train.Feature(bytes_list=tf.train.BytesList(value=[phrase])),
                'phrase_encoded': tf.train.Feature(int64_list=tf.train.Int64List(value=phrase_encoded)),
                })).SerializeToString()
    
    return record_bytes

def process_chunk(chunk, tfrecord_name):
    options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9)
    with tf.io.TFRecordWriter(tfrecord_name, options=options) as file_writer:
        for i, row in tqdm(chunk.iterrows()):
            record_bytes = encode_row(row)
            file_writer.write(record_bytes)
            del record_bytes
        file_writer.close()

In [11]:
row = train_df.iloc[0]

N_FILES = len(train_df)
CHUNK_SIZE = 512
N_PART = 1
FOLD = 4
part = 0

class CFG:
    seed = 42
    n_splits = 5

In [12]:
train_folds = train_df.copy()
train_folds['fold']=-1

train_folds = train_folds.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)

kfold = GroupKFold(n_splits=CFG.n_splits) 
print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds, groups=train_folds.participant_id)):
    train_folds.loc[valid_idx,'fold'] = fold_idx
    print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
    
assert not (train_folds['fold']==-1).sum()
assert len(np.unique(train_folds['fold']))==CFG.n_splits
train_folds.head()

5fold training 3933 samples
fold0: train 2523 valid 1410
fold1: train 3090 valid 843
fold2: train 3096 valid 837
fold3: train 3372 valid 561
fold4: train 3651 valid 282


,path,file_id,sequence_id,participant_id,phrase,fold
0,landmarks/102.parquet,102,20962,4,phantom,0
1,landmarks/102.parquet,102,20590,2,jakarta,1
2,landmarks/103.parquet,103,30108,1,disdain,3
3,landmarks/102.parquet,102,20259,1,manure,3
4,landmarks/103.parquet,103,31454,4,hungriest,0


In [13]:
%%time

import json
import os

DATASET_NAME = f'ausslfr-{CFG.n_splits}fold'

!rm -rf /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('/kaggle/input/kaggleapi/kaggle.json') as f:
    kaggle_creds = json.load(f)
    
os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

!kaggle datasets init -p /tmp/{DATASET_NAME}

with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)

dataset_meta['id'] = f'jiaweiliao1/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!cp /tmp/{DATASET_NAME}/dataset-metadata.json /tmp/{DATASET_NAME}/meta.json
!ls /tmp/{DATASET_NAME}

!kaggle datasets create -p /tmp/{DATASET_NAME} --public

Data package template written to: /tmp/ausslfr-5fold/dataset-metadata.json
{'title': 'ausslfr-5fold', 'id': 'jiaweiliao1/ausslfr-5fold', 'licenses': [{'name': 'CC0-1.0'}]}
dataset-metadata.json  meta.json
Starting upload for file meta.json
100%|██████████████████████████████████████████| 96.0/96.0 [00:00<00:00, 238B/s]
Upload successful: meta.json (96B)
Dataset creation error: The requested title "ausslfr-5fold" is already in use by a dataset. Please choose another title.
CPU times: user 64.2 ms, sys: 294 ms, total: 359 ms
Wall time: 7.49 s


In [14]:
import pandas as pd
import glob

# Path to your Parquet files
parquet_files_path = '/kaggle/input/australian-fingerspelling/landmarks/*.parquet'

# Initialize an empty list to store DataFrames
all_dataframes = []  # Change to all_dataframes for consistency

# Loop through each Parquet file
for file in glob.glob(parquet_files_path):
    df = pd.read_parquet(file)
    all_dataframes.append(df)  # Append to the correct list

# Concatenate all DataFrames into one
combined_df = pd.concat(all_dataframes, ignore_index=False)

# Count unique sequence_id based on the index
unique_sequences_count = combined_df.index.nunique()
print(f'Number of unique sequence_id: {unique_sequences_count}')

Number of unique sequence_id: 3933


In [15]:
# Put every image in a seperate TFRecord file
# Make Pairs of Views as input to the model

def split_dataframe(df, chunk_size=10000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size:(i + 1) * chunk_size])
    return chunks

for fold in range(CFG.n_splits):
    rows = train_folds[train_folds['fold'] == fold]
    print(f'Fold {fold}: {len(rows)} rows')
    chunks = split_dataframe(rows, CHUNK_SIZE)
    print(f'Fold {fold}: {len(chunks)} chunks created with sizes: {[len(chunk) for chunk in chunks]}')

    part_size = len(chunks) // N_PART
    last = (part + 1) * part_size if part != N_PART - 1 else len(chunks)  # Changed from len(chunks) + 1
    chunks = chunks[part * part_size:last]

    if not chunks:
        print(f"Empty chunks list for part {part} in fold {fold}")
        continue
    N = [len(x) for x in chunks]

    def safe_process_chunk(x, filepath):
        try:
            process_chunk(x, filepath)
        except KeyError as e:
            print(f"KeyError: {e} in fold {fold}, chunk with length {len(x)}")
        except Exception as e:
            print(f"Error: {e} in fold {fold}, chunk with length {len(x)}")

    _ = Parallel(n_jobs=cpu_count())(
        delayed(safe_process_chunk)(x, f'/tmp/{DATASET_NAME}/fold{fold}-{i}-{n}.tfrecords')
        for i, (x, n) in enumerate(zip(chunks, N))
    )


512it [23:34,  2.76s/it]
512it [23:47,  2.79s/it]
0it [00:00, ?it/s]
331it [12:52,  2.33s/it]
512it [19:51,  2.33s/it]
460it [17:18,  2.10s/it]
512it [19:03,  2.23s/it]
204it [07:25,  2.06s/it]
512it [17:57,  2.10s/it]
282it [09:08,  1.95s/it]

In [16]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

20241003-125828


In [17]:
!kaggle datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip

Starting upload for file fold2-1-325.tfrecords
100%|██████████████████████████████████████| 80.6M/80.6M [00:03<00:00, 28.1MB/s]
Upload successful: fold2-1-325.tfrecords (81MB)
Starting upload for file fold1-1-331.tfrecords
100%|██████████████████████████████████████| 90.1M/90.1M [00:03<00:00, 29.0MB/s]
Upload successful: fold1-1-331.tfrecords (90MB)
Starting upload for file fold0-1-512.tfrecords
100%|████████████████████████████████████████| 131M/131M [00:04<00:00, 31.5MB/s]
Upload successful: fold0-1-512.tfrecords (131MB)
Starting upload for file fold1-0-512.tfrecords
100%|████████████████████████████████████████| 146M/146M [00:04<00:00, 33.6MB/s]
Upload successful: fold1-0-512.tfrecords (146MB)
Starting upload for file fold3-0-512.tfrecords
100%|████████████████████████████████████████| 115M/115M [00:03<00:00, 31.8MB/s]
Upload successful: fold3-0-512.tfrecords (115MB)
Starting upload for file fold3-1-49.tfrecords
100%|██████████████████████████████████████| 11.2M/11.2M [00:01<00:00, 